In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('C:/Users/nerpa\Dropbox (Personal)\Research\discrete_sr\code\python\SPIDER_discrete')

import numpy as np

from commons.weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [ ]:
%%prun # profiling

dataset = 'data_bu.npy'
positions, vs, deltat, dims = load(dataset, 4)
deltat = float(deltat)
world_size = np.array([dims[0], dims[0], dims[1]])
print(world_size)
cg_res = 5
data_dict = {}
data_dict['v'] = vs
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
kernel_sigma = 0.6
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=kernel_sigma, cg_res=cg_res, deltat=deltat, cutoff=6)

# initialize libraries, domains, and weights
srd.make_libraries(max_complexity=5, max_observables=3, max_rho=2)

dom_width = 10
dom_time = 30
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=8*kernel_sigma)
srd.make_weights(m=16, qmax=1)
srd.make_library_matrices(debug=False)

save('Q_bu.npy', srd.dxs, srd.libs)

In [2]:
_, libs = load('Q_bu.npy', 2)
#_, libs = load('Q_XY_t0.01_eta0.1.npy', 2)
libs = libs.item()

In [5]:
from commons.sparse_reg_bf import *
import copy

# for regression we now need to construct a Scaler, Initializer, ModelIterator, and Threshold
scaler0 = Scaler(sub_inds=None, char_sizes=libs[0].col_weights, row_norms=None)
#init0 = Initializer(method='combinatorial', start_k=2)
init0 = Initializer(method='power', start_k=10)
#res0 = Residual(residual_type='fixed_column', anchor_col=0)
res0 = Residual(residual_type='dominant_balance')

iter0 = ModelIterator(max_k=10, backward_forward=True, brute_force=True) # test also boolean toggles
thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=None)
#thres0 = Threshold(threshold_type='information', ic=AIC)
#thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=3)

opts = {'scaler': scaler0, 'initializer': init0, 'residual': res0,
        'model_iterator': iter0, 'threshold': thres0}
opts['verbose'] = True
opts1 = copy.deepcopy(opts) # need to be careful to deep copy for the stateful ModelIterator
opts['inhomog'] = False
opts['inhomog_col'] = None
#opts['verbose'] = False
sub_inds1 = list(range(len(libs[0].terms)))
sub_inds1.remove(1)
opts1['scaler'] = Scaler(sub_inds=sub_inds1, char_sizes=libs[1].col_weights)
opts1['residual'] = Residual(residual_type='fixed_column', anchor_col=13)
    
opts1['inhomog'] = True
opts1['inhomog_col'] = 13 
    
print("###RANK 0 REGRESSION###")
Xi, lambd, best_term, lambda1 = sparse_reg_bf(libs[0].Q, **opts)
print("###RANK 1 REGRESSION###")
Xi2, lambd2, best_term2, lambda1_2 = sparse_reg_bf(libs[1].Q, **opts1)

for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}")
print("Model residual: ", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", libs[0].terms[best_term])

for i in range(len(Xi2)):
    if Xi2[i]!=0:
        print(f"[Term {i}] {Xi2[i]} * {libs[1].terms[i]}")
print("Model residual: ", lambd2)
print("One-term model residual:", lambda1_2)
print(f"Best term (#{best_term2}):", libs[1].terms[best_term2])

###RANK 0 REGRESSION###
Checking single-term residuals...
nrm[0]: 3871.6034738268945
nrm[1]: 4979.527604958296
nrm[2]: 2786.2223254672763
nrm[3]: 8333.698112780297
nrm[4]: 199.2416692221255
nrm[5]: 436.050821773103
nrm[6]: 45.977394213276064
nrm[7]: 128.96667441087567
nrm[8]: 11.496006535191352
nrm[9]: 4.597288376623669
nrm[10]: 16419.906145018016
nrm[11]: 19497.940288589136
nrm[12]: 494.11224613502173
nrm[13]: 1884.398438207465
nrm[14]: 257.53521355095785
nrm[15]: 54.83428265108655
nrm[16]: 1834.8076286449714
nrm[17]: 4015.5725325843696
nrm[18]: 423.4037690067376
nrm[19]: 105.86621055262937
nrm[20]: 4549.9577719333165
nrm[21]: 2371.7411464099187
nrm[22]: 10302.725238290914
nrm[23]: 6152.741590881443
nrm[24]: 3899.106388001571
nrm[25]: 614.4892250604215
nrm[26]: 180.0548731944538
nrm[27]: 25120.228606271066
nrm[28]: 32000.741162243175
nrm[29]: 1162.3631560362376
nrm[30]: 1046.0848308306242
nrm[31]: 1838.2106275104518
nrm[32]: 5658.803814676437
Initializing solution with starting k: 10


In [ ]:
### all unimplemented

from identify_models import *

eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=1e-3)